# Quantitative Assessment of Surgical Skills based on Muscular Activities

In [1]:
import torch
import numpy as np 
import scipy.io as sio
from os.path import dirname, join as pjoin
import matplotlib.pyplot as plt
from sklearn import preprocessing

### importing and formating the data from matlab : 

In [2]:
def exe_to_global_label(restimulus, inden):
    """
    this will take the label of ex2  or ex3 and convert it to a global label system
    the global label system is uniform for ex1,2 and 3 and goes from 0 to 52 
    """
    
    for i in range(len(restimulus)):
        if restimulus[i] != 0:
            restimulus[i] += inden 
        else:
            restimulus[i]=0
    return restimulus 


In [3]:
def get_data(n):
   
    """
    return the emg data and corresponding restimulus (for the 3 exos) of the n-th person
    """
    data_dir = pjoin('db_1', 'data', f's{n}')
    
    file_name_e1 = pjoin(data_dir, f'S{n}_A1_E1.mat')
    file_name_e2 = pjoin(data_dir, f'S{n}_A1_E2.mat')
    file_name_e3 = pjoin(data_dir, f'S{n}_A1_E3.mat')
    
    contents_e1 = sio.loadmat(file_name_e1)
    contents_e2 = sio.loadmat(file_name_e2)
    contents_e3 = sio.loadmat(file_name_e3)
    
    # those are np.array 
    emg_1 = contents_e1['emg']
    emg_2 = contents_e2['emg']
    emg_3 = contents_e3['emg']

    lbl_1 = contents_e1['restimulus']
    lbl_2 = contents_e2['restimulus']
    lbl_3 = contents_e3['restimulus']

    lbl_2 = exe_to_global_label(lbl_2,12) # so transforming the label into 13-29 (as 1-12 are already taken by ex_1)
    lbl_3 = exe_to_global_label(lbl_3,29) # so transforming the label into 30-52 (as 1-29are already taken by ex_1 and ex_2)  

    emg = np.concatenate((emg_1,emg_2,emg_3))
    labels = np.concatenate((lbl_1,lbl_2,lbl_3))
    
    return emg ,labels 

In [4]:
# concatenating the datas from all the subjects 
nb_subjects = 10 
emg= np.empty((0,10))
labels = np.empty((0,1))
for i in range(1,nb_subjects+1):
    emg_sub , labels_sub = get_data(i)
    emg = np.concatenate((emg,emg_sub))
    labels = np.concatenate((labels,labels_sub))
    
print(emg.shape)
print(labels.shape)

nb_channel = 10 

(4612696, 10)
(4612696, 1)


### separating the data by repetition : 
Think about the fact that the rest state will way more present than the other ones. Do we need to do something about it during training ? 

In [5]:
class repet():
    def __init__(self,emg,label):
        self.emg = emg
        self.label = label
        
        
class segment():
    
    def __init__(self,emg,label,nb_channel= 10):
        self.emg = emg
        self.label = label 
        self.nb_channel= nb_channel
        
    def mav(self):
        emg=self.emg
        nb_channel = self.nb_channel
        emg = np.absolute(emg)
        n  = emg.shape[0]
        mav=np.empty((1,nb_channel))
        for i in range(nb_channel):# for each channel
            mav[0][i] = (1/n)* np.sum(emg[i],axis = 0)
        return mav  
    
    def iemg(self):
        """
        emg=self.emg
        emg = np.absolute(emg)
        nb_channel = self.nb_channel
        iemg=np.empty((1,nb_channel))
        for i in range(nb_channel):# for each channel
            iemg[0][i] = np.sum(emg[i],axis = 0)
        return iemg
        """
        emg=self.emg
        emg = np.absolute(emg)
        nb_channel = self.nb_channel
        iemg=np.empty((1,nb_channel))
        for i in range(0,nb_channel):
            iemg[0][i]= np.sum(emg[i])
        return iemg
    
    def ssi(self):
        """
        emg=self.emg
        emg = np.absolute(emg)
        nb_channel = self.nb_channel
        ssi=np.empty((1,nb_channel))
        n  = emg.shape[0]
        for i in range(nb_channel): # for each channel
            for j in range(n):
                emg[j][i] = emg[j][i]*emg[j][i]
            ssi[0][i] = np.sum(emg[i],axis=0)
        return ssi
        """
        emg=self.emg
        emg = np.square(emg)
        nb_channel = self.nb_channel
        ssi=np.empty((1,nb_channel))
        for i in range(0,nb_channel):
            ssi[0][i]=np.sum(emg[i],axis=0)
        return ssi  
        
        
        
    def rms(self):
        """
        n  = emg.shape[0]  #number of samples 
        nb_channel = self.nb_channel
        rms=np.empty((1,nb_channel))
        for i in range(nb_channel):
            rms[0][i]=((1/n) * self.ssi()[0][i])**(0.5)
        return rms
        """
        emg=self.emg
        n  = emg.shape[0]  #number of samples 
        nb_channel = self.nb_channel
        rms=np.empty((1,nb_channel))
        emg = np.square(emg)
        
        for i in range(0,nb_channel):
            rms[0][i]= ((1/n)*np.sum(emg[i]))**(0.5)
        return rms 
        
   
    def var(self):
        """
        emg=self.emg
        n= emg.shape[0] #number of samples 
        nb_channel = self.nb_channel
        var = np.empty((1,nb_channel))
        
        for i in range(nb_channel):
            summation = 0
            mean_channel = np.mean(emg[:][i],axis= 0)
            for j in range(n):
                summation += (emg[j][i]-mean_channel)**2
            var[0][i] = (1/n)*summation
        """
        emg=self.emg
        nb_channel = self.nb_channel
        var = np.empty((1,nb_channel))
        for i in range(0,nb_channel):
            var[0][i]=np.var(emg[i])
        return var
    
    def ssc(self, x_thresh = 0.001): 
        emg=self.emg
        nb_channel = self.nb_channel
        ssc = np.empty((1,nb_channel))
        n= emg.shape[0] #number of samples 
        
        for i in range (nb_channel):
            f_i = 0  
            for j in range(1,n-1):
                cond_1 = emg[j][i]>emg[j-1][i] and emg[j][i]>emg[j+1][i]
                cond_2 = emg[j][i]<emg[j-1][i] and emg[j][i]<emg[j+1][i]
                cond_3 = abs(emg[j][i]- emg[j+1][i])> x_thresh
                cond_4 = abs(emg[j][i]- emg[j-1][i])> x_thresh
                if cond_1 or cond_2 or cond_3 or cond_4:
                    f_i += 1
            ssc[0][i] = (1/n)*f_i
        return ssc
    
    def zc(self, x_thresh = 0.0001): #zero crossings
        emg=self.emg
        nb_channel = self.nb_channel
        zc = np.empty((1,nb_channel))
        n= emg.shape[0] #number of samples 
        
        for i in range(nb_channel):
            f_i = 0 
            for j in range(n-1):
                cond_1 = emg[j][i]*emg[j+1][i]>0
                cond_2 = abs(emg[j][i] - emg[j+1][i])>x_thresh
                if cond_1 and cond_2 :
                    f_i += 1
                zc[0][i] = (1/n)*f_i
        return zc
      
    def get_features_by_channel(self,channel_nb):
        #return the all the features of this segment (for a given channel)
        feature_list = np.empty((0))
        
        feature_list = np.append(feature_list,self.mav()[0][channel_nb])
        feature_list = np.append(feature_list,self.iemg()[0][channel_nb])
        feature_list = np.append(feature_list,self.ssi()[0][channel_nb])
        feature_list = np.append(feature_list,self.rms()[0][channel_nb])
        feature_list = np.append(feature_list,self.var()[0][channel_nb])
        feature_list = np.append(feature_list,self.ssc()[0][channel_nb])
        feature_list = np.append(feature_list,self.zc()[0][channel_nb])
        
        
        
        return feature_list

In [6]:
def separate_repetition(emg,labels):
   
    """
    we have 53 possible class (52 movements and 1 for the rest state)
    
    input : - emg data of all the participant concatenated 
            - restimulus data , so the label corresponding to the emg data 
            
    output : - a 1-D list of repet elements
    """
    
    nb_samples = len(labels)
    
    print("Number of samples : " , nb_samples)
    list_repet = np.empty((0))
    
    accum = 0 
    current_lbl = 0 
    for i in range(nb_samples):
        if current_lbl == labels[i]:
            accum += 1
        else :
            diff = i - accum
            new_rep = repet(emg[diff:i],labels[i])
            
            list_repet = np.append(list_repet, new_rep)
            
            current_lbl = labels[i]
            accum = 0 
    return list_repet

In [7]:
list_repetition = separate_repetition(emg,labels)
print(list_repetition.shape)



Number of samples :  4612696
(10400,)


In [8]:
def segmentation(list_repet, window_type = 'adjacent', length = 256):
   
    """
    input : - the list a the emg data separated by repetition 
            - a param window_type (either adjacent or overlapped)(see "Myoelectric control systems— A survey")
            - the length of each window
    output : - a list of all the segments for all the exercice and all subjects
    """
    nb_repet = len(list_repet)
    print("We have " , nb_repet , "repetitions of excercices in the data")
    
    seg_data = np.empty((0))
    
    if window_type == 'adjacent': 
        
        for i in range(nb_repet):
            nb_windows = list_repet[i].emg.shape[0] // length #integer division
           
            for j in range(nb_windows): 
                emg_seg = list_repet[i].emg[j*length:(j+1)*length]
                label_seg = list_repet[i].label
                
                new_seg = segment(emg_seg,label_seg)
                seg_data = np.append(seg_data,new_seg) 
           
    if window_type == 'overlapped':
        print("Not implemented yet")
    print("Data shape : ", seg_data.shape)
    print(seg_data.shape)
    return seg_data
    

In [9]:
list_seg = segmentation(list_repetition)
print("shape : ", list_seg.shape)
print(list_seg[0].get_features_by_channel(9))

We have  10400 repetitions of excercices in the data
Data shape :  (12688,)
(12688,)
shape :  (12688,)
[0.0003707  0.0949     0.00293797 0.00338769 0.00020374 0.109375
 0.07421875]


# Classification part : 

In [10]:
from sklearn import svm
from sklearn.model_selection import train_test_split

In [ ]:
nb_feature = 7 

input_feature = np.empty((nb_channel,list_seg.shape[0],nb_feature))

l = list_seg.shape[0]
for i in range(0,nb_channel):
    print(i)
    for j in range(0,list_seg.shape[0]):
        print(j,"/",l)
        input_feature[i][j] = list_seg[j].get_features_by_channel(i)

In [12]:
label = np.empty((nb_channel,list_seg.shape[0]))
print(list_seg[0].label[0])
for i in range(0,nb_channel):
    print(i)
    for j in range(0,list_seg.shape[0]):
        label[i][j] = list_seg[j].label[0]
        
print("shape",label.shape)
print(input_feature[0].shape)
print(list_seg[11000].label[0])
print(list_seg.shape[0])
print(label)
print(nb_channel)

1.0
0
1
2
3
4
5
6
7
8
9
shape (10, 12688)
(12688, 7)
35.0
12688
[[ 1.  0.  1. ... 52.  0.  0.]
 [ 1.  0.  1. ... 52.  0.  0.]
 [ 1.  0.  1. ... 52.  0.  0.]
 ...
 [ 1.  0.  1. ... 52.  0.  0.]
 [ 1.  0.  1. ... 52.  0.  0.]
 [ 1.  0.  1. ... 52.  0.  0.]]
10


In [13]:
input_feature_all =np.empty((0,nb_feature))
label_all=np.empty((0))
for i in range(0,nb_channel):
    input_feature_all = np.append(input_feature_all, input_feature[i], axis=0) #combination off all 10 chanels
    label_all = np.append(label_all,label[i])
print(input_feature_all.shape)
print(label_all.shape)

for i in range(0,nb_channel):
    input_feature_all[:][:][i] = preprocessing.normalize([input_feature_all[:][:][i]])

"""
x_train= np.empty((0,nb_feature))
y_train= np.empty((0))

x_test= np.empty((0,nb_feature))
y_test= np.empty((0))

for i in range(0,nb_channel): # in this part , we study all the channels togheter
    x_train_temp, x_test_temp, y_train_temp, y_test_temp = train_test_split(input_feature[i],label[i], test_size=0.1, 
                                                        shuffle=True ) 
    print(x_train_temp.shape)
    print(y_train_temp.shape)
    x_train= np.append(x_train,x_train_temp,axis=0)
    y_train= np.append(y_train,y_train_temp,axis=0)
    
    x_test= np.append(x_test,x_test_temp,axis=0)
    y_test= np.append(y_test,y_test_temp,axis=0)

print(x_train.shape)
"""
x_train_all, x_test_all, y_train_all, y_test_all = train_test_split(input_feature_all,label_all, test_size=0.2, 
                                                        shuffle=True )


(126880, 7)
(126880,)


In [14]:
a = np.array([1, 2])
b = np.array([4,5,6])
print(a.shape)

a = np.append(a,b)
print(a.shape)

(2,)
(5,)


In [15]:
clf = svm.SVC(gamma='auto',kernel='rbf',class_weight='balanced',C=1.0, degree=3 ,max_iter= -1)
clf.fit(x_train_all, y_train_all)

SVC(class_weight='balanced', gamma='auto')

In [16]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

8383
33.03515132408575 % accuracy


In [17]:
print(100*compt/x_test_all.shape[0],"% accuracy")
print(clf.class_weight_)

33.03515132408575 % accuracy
[0.04689791 1.56340393 1.68144847 1.87946007 1.70692497 1.74582481
 1.80506109 1.8557847  1.89620773 1.6120958  1.74106346 1.58540547
 1.62440188 1.83094628 1.81360778 1.7442348  1.89808703 1.67703136
 1.60399482 1.60399482 1.85758469 2.09308176 1.83974045 1.73161827
 1.80847008 1.65672129 1.36602697 1.54823752 1.97033931 1.82397125
 1.76350811 1.32354514 1.79659457 1.30728315 1.63549941 1.58540547
 1.4541912  1.72227501 1.72537821 1.50091678 1.70086129 1.65672129
 1.74106346 1.50800773 1.6993521  1.66103193 1.73790364 1.6495864
 1.69634173 1.63271084 1.18293379 1.10384427 1.30283661]


In [18]:
#-------------------------------------------------------------
x_train, x_test, y_train, y_test = train_test_split(input_feature[0],label[0], test_size=0.2, 
                                                        shuffle=True )

In [19]:
clf = svm.SVC(gamma='auto',kernel='rbf',class_weight='balanced',C=1.0, degree=3 ,max_iter= 1)
clf.fit(x_train, y_train)

C:\Users\Ozgun\anaconda3\lib\site-packages\sklearn\svm\_base.py:284: ConvergenceWarning: Solver terminated early (max_iter=1).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


SVC(class_weight='balanced', gamma='auto', max_iter=1)

In [20]:
compt = 0
for i in range(0,x_test.shape[0]):
    if clf.predict([x_test[i]])[0] - y_test[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

40
0.15762925598991173 % accuracy


In [21]:
clf_linear = svm.SVC(gamma='auto',kernel='linear',class_weight='balanced',C=1.0, degree=3 ,max_iter= -1)
clf_linear.fit(x_train_all, y_train_all)

SVC(class_weight='balanced', gamma='auto', kernel='linear')

In [22]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_linear.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

8287
32.656841109709966 % accuracy


In [34]:
clf_sig = svm.SVC(gamma='auto',kernel='sigmoid',class_weight='balanced',C=1.0,max_iter= -1)
clf_sig.fit(x_train_all, y_train_all)

SVC(class_weight='balanced', gamma='auto', kernel='sigmoid')

In [35]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_sig.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

4757
18.746059268600252 % accuracy


In [24]:
from sklearn.ensemble import RandomForestClassifier

In [32]:
clf_rfc = RandomForestClassifier(max_depth=None,class_weight='balanced')
clf_rfc.fit(x_train_all, y_train_all)

RandomForestClassifier(class_weight='balanced')

In [33]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_rfc.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

9867
38.88319672131148 % accuracy


In [36]:
#-------------------- svm rbf increment C -----------------
c_values = np.array([0.1 , 0.5 , 1 , 5, 10,15, 20, 50,100,150,300,500,1000])

In [37]:
#----------------- MlP -------------------------
from sklearn.neural_network import MLPClassifier

In [38]:
clf_mlp = MLPClassifier(random_state=0, max_iter=300 , hidden_layer_sizes =(200,1),solver='adam', activation = 'logistic')
clf_mlp.fit(x_train_all, y_train_all)

C:\Users\Ozgun\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='logistic', hidden_layer_sizes=(200, 1), max_iter=300,
              random_state=0)

In [39]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10432
41.10970996216898 % accuracy


In [40]:
clf_mlp = MLPClassifier(random_state=0, max_iter=300 , hidden_layer_sizes =(400,1),solver='adam', activation = 'logistic')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(400, 1), max_iter=300,
              random_state=0)

In [41]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10428
41.093947036569986 % accuracy


In [42]:
clf_mlp = MLPClassifier(random_state=0, max_iter=300 , hidden_layer_sizes =(50,1),solver='adam', activation = 'logistic')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 1), max_iter=300,
              random_state=0)

In [43]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10432
41.10970996216898 % accuracy


In [48]:
clf_mlp = MLPClassifier(random_state=0, max_iter=300 , hidden_layer_sizes =(50,1),solver='adam', activation = 'logistic', learning_rate_init=0.01)
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 1),
              learning_rate_init=0.01, max_iter=300, random_state=0)

In [49]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10441
41.145176544766706 % accuracy


In [50]:
clf_mlp = MLPClassifier(random_state=0, max_iter=300 , hidden_layer_sizes =(50,1),solver='adam', activation = 'logistic', learning_rate_init=0.1)
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(50, 1),
              learning_rate_init=0.1, max_iter=300, random_state=0)

In [51]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10393
40.95602143757881 % accuracy


In [52]:
clf_mlp = MLPClassifier(random_state=0, max_iter=500 , hidden_layer_sizes =(200,1),solver='adam', activation = 'logistic')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(200, 1), max_iter=500,
              random_state=0)

In [53]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10433
41.11365069356873 % accuracy


In [56]:
clf_mlp = MLPClassifier(random_state=0, max_iter=500 , hidden_layer_sizes =(200,1),solver='adam', activation = 'tanh')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='tanh', hidden_layer_sizes=(200, 1), max_iter=500,
              random_state=0)

In [57]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10428
41.093947036569986 % accuracy


In [58]:
clf_mlp_2 = MLPClassifier(random_state=0, max_iter=500 , hidden_layer_sizes =(100,100),solver='adam', activation = 'logistic', learning_rate_init=0.01)
clf_mlp_2.fit(x_train_all, y_train_all)

MLPClassifier(activation='logistic', hidden_layer_sizes=(100, 100),
              learning_rate_init=0.01, max_iter=500, random_state=0)

In [59]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp_2.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10568
41.645649432534675 % accuracy


In [60]:
clf_mlp = MLPClassifier(random_state=0, max_iter=500 , hidden_layer_sizes =(200),solver='adam', activation = 'tanh')
clf_mlp.fit(x_train_all, y_train_all)

C:\Users\Ozgun\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:692: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (500) reached and the optimization hasn't converged yet.
  warnings.warn(


MLPClassifier(activation='tanh', hidden_layer_sizes=200, max_iter=500,
              random_state=0)

In [61]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10493
41.35009457755359 % accuracy


In [62]:
clf_mlp = MLPClassifier(random_state=0, max_iter=1000 , hidden_layer_sizes =(200),solver='adam', activation = 'tanh')
clf_mlp.fit(x_train_all, y_train_all)

MLPClassifier(activation='tanh', hidden_layer_sizes=200, max_iter=1000,
              random_state=0)

In [63]:
compt = 0
for i in range(0,x_test_all.shape[0]):
    if clf_mlp.predict([x_test_all[i]])[0] - y_test_all[i] == 0.0:
        compt +=1
print(compt)

print(100*compt/x_test_all.shape[0],"% accuracy")

10536
41.51954602774275 % accuracy
